## PyTorch Resnets

In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(
    project_dir, '..', '..', '..', '..')))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH
)
from experiments.utils.parser import Parser

In [18]:
series = 201
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = 'resnet-human'
series_path = os.path.join(
    NODE_PROFILING_RESULTS_PATH,
    'series',
    str(series))
loader = Loader(
    series_path=series_path, config_key_mapper=config_key_mapper,
    model_name=model_name)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [19]:
# results = loader.read_results(selected)
# final_dataframe = []
# for experiment_id, result in results.items():
#     processed_exp = {'experiment_id': int(experiment_id)}
#     flattened_results = self.flatten_results(
#         results[str(experiment_id)]['responses'])
#     if log is not None:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log[experiment_id])
#     else:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log)     

In [20]:
results.columns

Index(['experiment_id', 'client_to_model_latencies_avg',
       'client_to_model_latencies_p99', 'client_to_model_latencies_p50',
       'client_to_model_latencies_var', 'client_to_model_latencies_max',
       'client_to_model_latencies_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_latencies_avg', 'model_to_client_latencies_p99',
       'model_to_client_latencies_p50', 'model_to_client_latencies_var',
       'model_to_client_latencies_max', 'model_to_client_latencies_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_coun

In [21]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config) 

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1'],
    'data_type': 'image',
    'max_batch_size': ['8'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['resnet18'],
    'node_name': 'resnet-human',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 201,
    'series_meta': 'all variables change for making the profiler 1,\\n cpu '
                   'type: Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 30, 'loads_to_test': [40]},
    'workload_type': 'static'}
File name: 1.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1'],
    'data_type': 'image',
    'max_batch_size': ['2'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['resnet18'],
    'node_name': 'r

In [22]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,resnet-human,resnet18,1,10Gi,8,1,40,30,201,all variables change for making the profiler 1...,1,False,exponential,image,1
1,2,video,resnet-human,resnet18,1,10Gi,2,1,40,30,201,all variables change for making the profiler 1...,1,False,exponential,image,1
2,3,video,resnet-human,resnet18,1,10Gi,2,1,1,30,201,all variables change for making the profiler 1...,1,False,exponential,image,1
3,4,video,resnet-human,resnet18,1,10Gi,2,1,20,30,201,all variables change for making the profiler 1...,1,False,exponential,image,1


In [6]:
experiment_ids = key_config_df[
    (key_config_df['model_variant'] == 'resnet18') & (key_config_df['cpu_request'] == 1)]['experiment_id'].tolist()
metadata_columns = ['model_variant', 'max_batch_size', 'cpu_request', 'load']
results_columns = ['model_latencies_min', 'model_latencies_p99', 'cpu_usage_count_avg', 'model_latencies_avg', 'throughput_max']
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns)
output['expected_throughput'] = 1/output['model_latencies_avg'] * output['max_batch_size']
# output.sort_values(by='throughput')
display(output)

,model_variant,max_batch_size,cpu_request,load,model_latencies_min,model_latencies_p99,cpu_usage_count_avg,model_latencies_avg,throughput_max,expected_throughput
0,resnet18,8,1,40,0.099253,0.402170,14.506061,0.291324,23.469388,27.460874
1,resnet18,2,1,40,0.073981,0.136444,9.888050,0.086079,22.200027,23.234588


In [7]:
results

,experiment_id,client_to_model_latencies_avg,client_to_model_latencies_p99,client_to_model_latencies_p50,client_to_model_latencies_var,client_to_model_latencies_max,client_to_model_latencies_min,model_latencies_avg,model_latencies_p99,model_latencies_p50,...,memory_usage_p50,memory_usage_var,memory_usage_max,memory_usage_min,throughput_avg,throughput_p99,throughput_p50,throughput_var,throughput_max,throughput_min
0,1,11.446951,21.287707,11.778314,36.554491,22.548206,0.07395,0.291324,0.402170,0.300667,...,310288384.0,4.157590e+16,685236224.0,4321280.0,10.249094,23.345849,9.874538,57.888059,23.469388,0.077922
1,2,13.070138,24.487117,13.381670,47.263722,24.916646,0.05088,0.086079,0.136444,0.083140,...,310849536.0,2.436896e+16,472645632.0,4407296.0,9.979404,21.958282,9.746301,48.279323,22.200027,0.099093
